## Libraries

In [1]:
import polars as pl
from pathlib import Path
from datetime import datetime

## Paths

In [2]:
root_dir = Path("..")
data_dir = root_dir / "data/m5"
processed_data_dir = data_dir / "processed"

processed_data_dir.mkdir(exist_ok=True)

## Prepare data

In [3]:
sales_df = pl.read_csv(data_dir / "sales_train_evaluation.csv")
prices_df = pl.read_csv(data_dir / "sell_prices.csv")
calendar_df = pl.read_csv(data_dir / "calendar.csv")

hierarchy_df = (
    sales_df
    .rename({"id": "unique_id"})
    .select(["unique_id", "item_id", "dept_id", "cat_id", "store_id", "state_id"])
    .with_columns(
        pl.col("unique_id").str.replace("_evaluation", "")
    )
)

calendar_df = (
    calendar_df
    .with_columns(
        pl.col("date").cast(pl.datatypes.Date),
        pl.col("d").str.replace("d_", "").cast(pl.datatypes.Int64)
    )
    .drop("weekday")
    .rename({"d": "i"})
)

sales_df = (
    sales_df
    .rename({"id": "unique_id"})
    .with_columns(
        pl.col("unique_id").str.replace("_evaluation", "")
    )
    .drop(["item_id", "dept_id", "cat_id", "store_id", "state_id"])
    .unpivot(index=["unique_id"], variable_name="i", value_name="y")
    .with_columns(
        pl.col("i").str.replace("d_", "").cast(pl.datatypes.Int64)
    )
    .join(hierarchy_df.select("unique_id", "item_id", "store_id"), on="unique_id", how="left")
    .join(calendar_df, on="i", how="left")
    .join(prices_df, on=["item_id", "store_id", "wm_yr_wk"], how="left")
    .drop("item_id", "store_id")
)

In [4]:
sales_df

unique_id,i,y,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
str,i64,i64,date,i64,i64,i64,i64,str,str,str,str,i64,i64,i64,f64
"""HOBBIES_1_001_CA_1""",1,0,2011-01-29,11101,1,1,2011,null,null,null,null,0,0,0,null
"""HOBBIES_1_002_CA_1""",1,0,2011-01-29,11101,1,1,2011,null,null,null,null,0,0,0,null
"""HOBBIES_1_003_CA_1""",1,0,2011-01-29,11101,1,1,2011,null,null,null,null,0,0,0,null
"""HOBBIES_1_004_CA_1""",1,0,2011-01-29,11101,1,1,2011,null,null,null,null,0,0,0,null
"""HOBBIES_1_005_CA_1""",1,0,2011-01-29,11101,1,1,2011,null,null,null,null,0,0,0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""FOODS_3_823_WI_3""",1941,1,2016-05-22,11617,2,5,2016,null,null,null,null,0,0,0,2.98
"""FOODS_3_824_WI_3""",1941,0,2016-05-22,11617,2,5,2016,null,null,null,null,0,0,0,2.48
"""FOODS_3_825_WI_3""",1941,2,2016-05-22,11617,2,5,2016,null,null,null,null,0,0,0,3.98


In [5]:
hierarchy_df

unique_id,item_id,dept_id,cat_id,store_id,state_id
str,str,str,str,str,str
"""HOBBIES_1_001_CA_1""","""HOBBIES_1_001""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA"""
"""HOBBIES_1_002_CA_1""","""HOBBIES_1_002""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA"""
"""HOBBIES_1_003_CA_1""","""HOBBIES_1_003""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA"""
"""HOBBIES_1_004_CA_1""","""HOBBIES_1_004""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA"""
"""HOBBIES_1_005_CA_1""","""HOBBIES_1_005""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA"""
…,…,…,…,…,…
"""FOODS_3_823_WI_3""","""FOODS_3_823""","""FOODS_3""","""FOODS""","""WI_3""","""WI"""
"""FOODS_3_824_WI_3""","""FOODS_3_824""","""FOODS_3""","""FOODS""","""WI_3""","""WI"""
"""FOODS_3_825_WI_3""","""FOODS_3_825""","""FOODS_3""","""FOODS""","""WI_3""","""WI"""


In [6]:
sales_df.write_parquet(processed_data_dir / "sales_df.parquet")
hierarchy_df.write_parquet(processed_data_dir / "hierarchy_df.parquet")